### construct H matrix, relationship matrix for both genotyped and non-genotyped individuals

* Input:
 * genotype file (missing value denoted as NA) without centering 
 * pedigree file (all animals in genotype files are in pedigree file)

* output
 * relationship matrix for both genotyped and non-genotyped individuals

In [ ]:
##Data in Box/Data

In [1]:
using JWAS:SSBR,misc,PedModule
using DataFrames,CSV

In [2]:
PATH="./";

#### 1. get genotype file and pedigree file ready 

* remove header in pedigree file and change seperator to ' '

In [8]:
pedfile=PATH*"Trial16and20_Pedigree.csv"
temp=CSV.read(pedfile);
CSV.write(PATH*"pedfile.txt",temp,header=false,delim=' ');

* genotype,remove header, repalce missing value with column means

In [9]:
genofile=PATH*"AllPigs_not_centered_SingleStep.txt";

In [ ]:
temp=readtable(genofile,header=true,separator=' ');
#temp=CSV.read(genofile,header=true,delim=' ',null="NA");

In [ ]:
writetable(PATH*"genofile.txt",temp,header=false);

In [ ]:
ncol= size(temp,2)
etv = Array(DataType,ncol)
fill!(etv,Float64)
etv[1]=String;

In [ ]:
temp=readtable(PATH*"genofile.txt",header=false,separator=',',eltypes=etv);

> functions to repalce missing values with column mean 

In [ ]:
function missing2mean!(X::DataFrames.DataFrame;id4row=true)
    nrow,ncol = size(X)
    start = 1
    if id4row==true
      start+=1
    end
    for i=start:ncol
        index=find(x->typeof(x)==Missings.Missing,X[:,i])
        cols = collect(1:nrow)
        deleteat!(cols,index)
        #X[index,i]=round(Int,mean(X[cols,i]))
        X[index,i]=mean(X[cols,i])
    end
end

In [ ]:
typeof(temp[1,2])==Missings.Missing

In [ ]:
missing2mean!(temp);

In [ ]:
writetable(PATH*"genofile.txt",temp,header=false,separator=' ');

#### 2. construct H matrix

In [ ]:
num  =SSBR.Numbers(0,0,0,0,0,0,0);

In [ ]:
genofile=PATH*"genofile.txt"
geno = misc.make_genotypes(genofile,header=false,center=true);

In [ ]:
fieldnames(geno)

In [ ]:
ped,amats =SSBR.calc_Ai(PATH*"pedfile.txt",geno,num);

In [ ]:
mmats = SSBR.make_MMats(geno,num,amats,ped);

In [ ]:
M = mmats.full
H = M*M'/geno.sum2pq;

In [ ]:
res=inv(full(amats.nn));

In [ ]:
H[1:num.pedn,1:num.pedn]=H[1:num.pedn,1:num.pedn]+res;

In [ ]:
writedlm("Hmatrix.txt",H)

In [ ]:
mean(diag(H))

In [ ]:
size(H,1)

In [ ]:
IDs=PedModule.getIDs(ped);

In [ ]:
sum(IDs.== IDs)

In [ ]:
out=convert(DataFrame, H);

In [ ]:
H

In [ ]:
temp=hcat(IDs,H)
header=["H_matrix"; IDs]
out=vcat(reshape(header,1,length(header)),temp);

In [ ]:
writedlm("Hmatrix.txt",out)

In [ ]:
# find missing ID
a=Set(pedid)
b=Set(genoid)
c=intersect(a, b)
setdiff(b,c)